# ====== 03.31 수 ========= pdf참고

# Boosting
부스팅(Boosting)이란 단순하고 약한 학습기(Weak Learner)들를 결합해서 보다 정확하고 강력한 학습기(Strong Learner)를 만드는 방식.  
정확도가 낮은 하나의 모델을 만들어 학습 시킨뒤, 그 모델의 예측 오류는 두 번째 모델이 보완한다. 이 두 모델을 합치면 처음보다는 정확한 모델이 만들어 진다. 합쳐진 모델의 예측 오류는 다음 모델에서 보완하여 계속 더하는 과정을 반복한다.

- 약한 학습기들은 앞 학습기가 만든 오류를 줄이는 방향으로 학습한다.
- gradient boosting
    - 처음 모델은 y를 예측 두번째 부터는 앞 모델이 만든 오류를 예측 그것을 앞 모델에 업데이트하면 오류를 줄일 수 있다.
    - 그 오류를 update할 때 뺄까 더할까를 gradient descent 방법을 쓴다. 미분해서 나오는 값의 음수를 취해서 적용. 
    - 학습률을 작게하면 update가 조금씩 크면 많이 하게 된다. 그래서 크게하면 학습데이터에 너무 맞아 과대적합 될 수 있다.

## GradientBoosting
- 개별 모델로 Decision Tree 를 사용한다. 
- depth가 깊지 않은 트리를 많이 연결해서 이전 트리의 오차를 보정해 나가는 방식으로 실행한다.
- 오차를 보정할 때 경사하강법(Gradient descent)을 사용한다.
- 얕은 트리를 많이 연결하여 각각의 트리가 데이터의 일부에 대해 예측을 잘 수행하도록 하고 그런 트리들이 모여 전체 성능을 높이는 것이 기본 아이디어.
- 분류와 회귀 둘다 지원하는 모델 (GradientBoostingClassification, GrandientBoostingRegressor)
- 훈련시간이 많이 걸리고, 트리기반 모델의 특성상 희소한 고차원 데이터에서는 성능이 않좋은 단점이 있다.
    - 희소한 고차원 --> 희소한 : 값들에 0이 많은 / 고차원 : 컬럼이 많은 

### 주요 파라미터
- Decision Tree 의 가지치기 관련 매개변수
    - 각각의 tree가 복잡한 모델이 되지 않도록 한다. 
- learning rate
    - 이전 tree의 오차를 얼마나 강하게 보정할 것인지 제어하는 값. 
    - 값이 크면 보정을 강하게 하여 복잡한 모델을 만든다. 학습데이터의 정확도는 올라가지만 과대적합이 날 수있다. 
    - 값을 작게 잡으면 보정을 약하게 하여 모델의 복잡도를 줄인다. 과대적합을 줄일 수 있지만 성능 자체가 낮아질 수있다.
    - 기본값 : 0.1
- n_estimators
    - tree의 개수 지정. 많을 수록 복잡한 모델이 된다.
- n_iter_no_change, validation_fraction
    - validation_fraction에 지정한 비율만큼 n_iter_no_change에 지정한 반복 횟수동안 검증점수가 좋아 지지 않으면 훈련을 조기 종료한다.

- 보통 max_depth를 낮춰 개별 트리의 복잡도를 낮춘다. (5가 넘지 않게) 그리고 n_estimators를 가용시간, 메모리 한도에 맞춘뒤 적절한 learning_rate을 찾는다.

$ 필기 $  
- 대용량(행, 컬럼 양이 많은) 일때 성능이 좋다. 


In [24]:
!pip install xgboost

In [1]:
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import accuracy_score

In [3]:
# data 
data = load_breast_cancer()
X, y = data['data'], data['target']
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, random_state=1)
y_train.shape # 이정도 양은 그래디언트부스팅을 사용하기에 적합한 수준의 양은 아니다. 양이 많아야 좋음 
# 트리모델이기때문에 라벨인코딩 등 처리하면 된다. 

# model 
gb = GradientBoostingClassifier(random_state=1)
gb.fit(X_train, y_train)

# predict
pred_train = gb.predict(X_train)
pred_test = gb.predict(X_test)

accuracy_score(y_train, pred_train), accuracy_score(y_test, pred_test) 

GradientBoostingClassifier(random_state=1)

In [5]:
# 트리모델이기 때문에 피처의 중요도 확인 가능 
import pandas as pd 
fi = gb.feature_importances_

fi_s = pd.Series(fi, index = data['feature_names'])
fi_s.sort_values(ascending=False)

worst radius               0.383871
worst concave points       0.285990
worst perimeter            0.130654
mean concave points        0.046544
worst area                 0.042472
worst texture              0.041187
worst concavity            0.012790
area error                 0.010906
mean texture               0.009120
mean concavity             0.007964
radius error               0.004788
concavity error            0.003404
worst fractal dimension    0.002838
worst symmetry             0.002546
mean area                  0.002540
fractal dimension error    0.002532
mean compactness           0.001572
compactness error          0.001384
mean perimeter             0.001381
smoothness error           0.001340
symmetry error             0.001267
perimeter error            0.000813
mean radius                0.000656
mean fractal dimension     0.000625
texture error              0.000575
worst compactness          0.000097
worst smoothness           0.000066
mean symmetry              0

### GridSearchCV 이용해 최적의 하이퍼파라미터 찾기

In [11]:
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV

param = {
    'n_estimators' : [100, 200, 300, 400, 500], # tree 갯수 (defaul : 100)
    'learning_rate' : [0.001, 0.005, 0.01, 0.1], # 학습률 
    'max_depth' : range(1,5),
    'subsample' : [0.5, 0.7, 1], # 학습시킬 샘플의 비율
}
gb = GradientBoostingClassifier(random_state=1)
gs = GridSearchCV(gb,
                  param_grid = param,
                  cv = 3, 
                  scoring='accuracy',
                  n_jobs = -1,
)

In [12]:
gs.fit(X_train, y_train)

GridSearchCV(cv=3, estimator=GradientBoostingClassifier(random_state=1),
             n_jobs=-1,
             param_grid={'learning_rate': [0.001, 0.005, 0.01, 0.1],
                         'max_depth': range(1, 5),
                         'n_estimators': [100, 200, 300, 400, 500],
                         'subsample': [0.5, 0.7, 1]},
             scoring='accuracy')

In [14]:
gs.best_params_

{'learning_rate': 0.1, 'max_depth': 2, 'n_estimators': 200, 'subsample': 0.5}

In [16]:
result_df = pd.DataFrame(gs.cv_results_)
result_df.sort_values('rank_test_score').head(5)

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_learning_rate,param_max_depth,param_n_estimators,param_subsample,params,split0_test_score,split1_test_score,split2_test_score,mean_test_score,std_test_score,rank_test_score
203,0.927302,0.006951,0.001932,0.000010,0.1,2,300,1,"{'learning_rate': 0.1, 'max_depth': 2, 'n_esti...",0.957746,0.971831,0.992958,0.974178,0.014470,1
198,0.425580,0.003506,0.001655,0.000112,0.1,2,200,0.5,"{'learning_rate': 0.1, 'max_depth': 2, 'n_esti...",0.971831,0.971831,0.978873,0.974178,0.003320,1
229,0.885066,0.002948,0.002238,0.000202,0.1,4,200,0.7,"{'learning_rate': 0.1, 'max_depth': 4, 'n_esti...",0.964789,0.971831,0.985915,0.974178,0.008783,1
206,1.223143,0.003071,0.002308,0.000106,0.1,2,400,1,"{'learning_rate': 0.1, 'max_depth': 2, 'n_esti...",0.964789,0.971831,0.985915,0.974178,0.008783,1
204,0.849993,0.001701,0.002248,0.000077,0.1,2,400,0.5,"{'learning_rate': 0.1, 'max_depth': 2, 'n_esti...",0.964789,0.971831,0.978873,0.971831,0.005750,5


In [17]:
# predict 
pred_test = gs.predict(X_test)
accuracy_score(y_test,pred_test)

0.951048951048951

In [18]:
# feature_importances_ 확인하기. 그리드서치는 ?가 없어 ? 
model = gs.best_estimator_
model.feature_importances_

array([0.00158211, 0.01853285, 0.00143937, 0.01113539, 0.00046681,
       0.00342275, 0.0032202 , 0.07233424, 0.00441985, 0.00195512,
       0.00912057, 0.01049067, 0.00278726, 0.04558971, 0.00395389,
       0.00230444, 0.0079779 , 0.00140281, 0.00380318, 0.00751886,
       0.18147091, 0.02102962, 0.22247248, 0.06252418, 0.00042761,
       0.00888623, 0.01203544, 0.24964542, 0.02643172, 0.00161843])

- 그리디언트 부스팅의 단점을 보완한 모델 
- 어떤 단점? 어떤 개선점? 시간이 느리다. 학습시간이 좀 걸린다. / 과적합을 제어할수있는 규제를 제공 
- 그래디언트 부스팅보다 빠른거지 XG부스트또한 학습속도가 걸리긴 하다. 

# XGBoost(Extra Gradient Boost)
- https://xgboost.readthedocs.io/
- Gradient Boost 알고리즘을 기반으로 개선해서 나온 모델.
- 캐글 경진대회에서 상위에 입상한 데이터 과학자들이 사용한 것을 알려저 유명해짐.
- Gradient Boost의 단점인 느린수행시간을 해결하고 과적합을 제어할 수 있는 규제를 제공하여 성능을 높임.
- 두가지 개발 방법
    - [Scikit-learn 래퍼 XGBoost 모듈 사용](https://xgboost.readthedocs.io/en/latest/python/python_api.html#module-xgboost.sklearn)
    - [파이썬 래퍼 XGBoost 모듈 사용](https://xgboost.readthedocs.io/en/latest/python/python_api.html#module-xgboost.training)
- 설치   
``
pip install xgboost
conda install -y -c anaconda py-xgboost
``



In [26]:
!conda install -y -c anaconda py-xgboost

Solving environment: failed with initial frozen solve. Retrying with flexible solve.
Solving environment: failed with repodata from current_repodata.json, will retry with next repodata source.
Solving environment: failed with initial frozen solve. Retrying with flexible solve.
Solving environment: \ 
Found conflicts! Looking for incompatible packages.
This can take several minutes.  Press CTRL-C to abort.
                                                                               failed

UnsatisfiableError: The following specifications were found
to be incompatible with the existing python installation in your environment:

Specifications:

  - py-xgboost -> python[version='>=2.7,<2.8.0a0|>=3.6,<3.7.0a0|>=3.7,<3.8.0a0|>=3.5,<3.6.0a0']

Your python: python=3.8

If python is on the left-most side of the chain, that's the version you've asked for.
When python appears to the right, that indicates that the thing on the left is somehow
not available for the python version you are constrai

## Scikit-learn 래퍼 XGBoost
- XGBoost를 Scikit-learn프레임워크와 연동할 수 있도록 개발됨.
- Scikit-learn의 Estimator들과 동일한 패턴으로 코드를 작성할 수 있다.
- GridSearchCV나 Pipeline 등 Scikit-learn이 제공하는 다양한 유틸리티들을 사용할 수 있다.
- XGBClassifier: 분류
- XGBRegressor : 회귀 

### 주요 매개변수
- learning_rate : 학습률, 보통 0.01 ~ 0.2 사이의 값 사용
- n_estimators : week tree 개수
- max_depth: 트리의 depth 지정.

In [25]:
from xgboost import XGBClassifier

xgb = XGBClassifier(n_estimators = 200,
                    learning_rate = 0.5,
                    max_depth = 2,
                    random_state =1
)
xgb.fit(X_train, y_train)

XGBoostError: XGBoost Library (libxgboost.dylib) could not be loaded.
Likely causes:
  * OpenMP runtime is not installed (vcomp140.dll or libgomp-1.dll for Windows, libomp.dylib for Mac OSX, libgomp.so for Linux and other UNIX-like OSes). Mac OSX users: Run `brew install libomp` to install OpenMP runtime.
  * You are running 32-bit Python on a 64-bit OS
Error message(s): ['dlopen(/opt/anaconda3/lib/python3.8/site-packages/xgboost/lib/libxgboost.dylib, 6): Library not loaded: /usr/local/opt/libomp/lib/libomp.dylib\n  Referenced from: /opt/anaconda3/lib/python3.8/site-packages/xgboost/lib/libxgboost.dylib\n  Reason: image not found']


In [ ]:
pred_train = xgb.predict(X_train)
pred_test = xgb.predict(X_test)

accuracy_score(y_train, pred_train), accuracy_score(y_test, pred_test)

In [21]:
fi = xgb.feature_importances_


NameError: name 'xgb' is not defined